In [1]:
import bert
import codecs
import collections
from bert import tokenization
from bert import modeling
import tensorflow as tf
import numpy as np
from bert.extract_features import InputExample, convert_examples_to_features, model_fn_builder, input_fn_builder

In [2]:
max_seq_len = 30
pred_batch_size = 32
init_checkpoint = "multi_cased_L-12_H-768_A-12/bert_model.ckpt"
config_path = "multi_cased_L-12_H-768_A-12/bert_config.json"
vocab_path = "multi_cased_L-12_H-768_A-12/vocab.txt"
layer_indexes = [-1,-2,-3,-4]

In [3]:
bert_config = modeling.BertConfig.from_json_file(config_path)
tokenizer = tokenization.FullTokenizer(
      vocab_file=vocab_path, do_lower_case=False)
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.contrib.tpu.RunConfig(
    master=None,
    tpu_config=tf.contrib.tpu.TPUConfig(
    num_shards=1,
    per_host_input_for_training=is_per_host))
sess = tf.Session()

In [4]:
examples = [InputExample(0, "Трудно понять наши цели", None)]

In [5]:
# features = convert_examples_to_features(
#       examples=examples, seq_length=max_seq_len, tokenizer=tokenizer)
# unique_id_to_feature = {}
# for feature in features:
#     unique_id_to_feature[feature.unique_id] = feature
#     print(feature)

In [6]:
def convert_examples_to_features(examples, seq_length, tokenizer):
  """Loads a data file into a list of `InputBatch`s."""

  features = []
    
  input_ids = np.zeros((len(examples), seq_length), dtype=np.int32)
  input_mask = np.zeros((len(examples), seq_length), dtype=np.int32)
    
  for (ex_index, example) in enumerate(examples):
    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
      tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
      # Modifies `tokens_a` and `tokens_b` in place so that the total
      # length is less than the specified length.
      # Account for [CLS], [SEP], [SEP] with "- 3"
      _truncate_seq_pair(tokens_a, tokens_b, seq_length - 3)
    else:
      # Account for [CLS] and [SEP] with "- 2"
      if len(tokens_a) > seq_length - 2:
        tokens_a = tokens_a[0:(seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    input_type_ids = []
    tokens.append("[CLS]")
    input_type_ids.append(0)
    for token in tokens_a:
      tokens.append(token)
      input_type_ids.append(0)
    tokens.append("[SEP]")
    input_type_ids.append(0)

    if tokens_b:
      for token in tokens_b:
        tokens.append(token)
        input_type_ids.append(1)
      tokens.append("[SEP]")
      input_type_ids.append(1)

    token_ids = np.array(tokenizer.convert_tokens_to_ids(tokens))
    input_ids[ex_index, :token_ids.size] = token_ids
    input_mask[ex_index, :token_ids.size] = 1

  return input_ids, input_mask

#     # The mask has 1 for real tokens and 0 for padding tokens. Only real
#     # tokens are attended to.
#     input_mask = [1] * len(input_ids)

#     # Zero-pad up to the sequence length.
#     while len(input_ids) < seq_length:
#       input_ids.append(0)
#       input_mask.append(0)
#       input_type_ids.append(0)

#     assert len(input_ids) == seq_length
#     assert len(input_mask) == seq_length
#     assert len(input_type_ids) == seq_length

#     if ex_index < 5:
#       tf.logging.info("*** Example ***")
#       tf.logging.info("unique_id: %s" % (example.unique_id))
#       tf.logging.info("tokens: %s" % " ".join(
#           [tokenization.printable_text(x) for x in tokens]))
#       tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
#       tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
#       tf.logging.info(
#           "input_type_ids: %s" % " ".join([str(x) for x in input_type_ids]))

#     features.append(
#         InputFeatures(
#             unique_id=example.unique_id,
#             tokens=tokens,
#             input_ids=input_ids,
#             input_mask=input_mask,
#             input_type_ids=input_type_ids))
#   return features

In [7]:
ids, mask = convert_examples_to_features(
      examples=examples, seq_length=max_seq_len, tokenizer=tokenizer)

In [9]:
tokenizer.convert_tokens_to_ids(['[CLS]'])

[101]

In [37]:
def built_bert(bert_config,
           init_checkpoint,
           use_one_hot_embeddings=False,
           seq_len=30):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = tf.placeholder(shape=(None, seq_len), dtype=tf.int32, name="input_ids")
    input_mask = tf.placeholder(shape=(None, seq_len), dtype=tf.int32, name="input_mask")
    input_type_ids = None

    model = modeling.BertModel(
        config=bert_config,
        is_training=False,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=input_type_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    tvars = tf.trainable_variables()
    scaffold_fn = None
    (assignment_map,
     initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(
         tvars, init_checkpoint)

    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    all_layers = model.get_all_encoder_layers()

    return model, all_layers, input_ids, input_mask

In [42]:
bert_model, layers, input_ids_tf, input_mask_tf = built_bert(
      bert_config=bert_config,
      init_checkpoint=init_checkpoint)
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = fgdfdfdsfwdfsg:0, shape = (10, 20)
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (119547, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/key/bias:0, 

INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflo

INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKP

INFO:tensorflow:  name = bert_1/encoder/layer_1/intermediate/dense/kernel:0, shape = (768, 3072)
INFO:tensorflow:  name = bert_1/encoder/layer_1/intermediate/dense/bias:0, shape = (3072,)
INFO:tensorflow:  name = bert_1/encoder/layer_1/output/dense/kernel:0, shape = (3072, 768)
INFO:tensorflow:  name = bert_1/encoder/layer_1/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_1/encoder/layer_1/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert_1/encoder/layer_1/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert_1/encoder/layer_2/attention/self/query/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_1/encoder/layer_2/attention/self/query/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_1/encoder/layer_2/attention/self/key/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_1/encoder/layer_2/attention/self/key/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_1/encoder/layer_2/attention/self/value/kernel:0, shape

INFO:tensorflow:  name = bert_1/encoder/layer_7/attention/self/key/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_1/encoder/layer_7/attention/self/key/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_1/encoder/layer_7/attention/self/value/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_1/encoder/layer_7/attention/self/value/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_1/encoder/layer_7/attention/output/dense/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_1/encoder/layer_7/attention/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_1/encoder/layer_7/attention/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert_1/encoder/layer_7/attention/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert_1/encoder/layer_7/intermediate/dense/kernel:0, shape = (768, 3072)
INFO:tensorflow:  name = bert_1/encoder/layer_7/intermediate/dense/bias:0, shape = (3072,)
INFO:tensorflow:  name = bert_1/encoder/layer

INFO:tensorflow:  name = bert_2/encoder/layer_0/attention/self/value/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_2/encoder/layer_0/attention/self/value/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_0/attention/output/dense/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_2/encoder/layer_0/attention/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_0/attention/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_0/attention/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_0/intermediate/dense/kernel:0, shape = (768, 3072)
INFO:tensorflow:  name = bert_2/encoder/layer_0/intermediate/dense/bias:0, shape = (3072,)
INFO:tensorflow:  name = bert_2/encoder/layer_0/output/dense/kernel:0, shape = (3072, 768)
INFO:tensorflow:  name = bert_2/encoder/layer_0/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_0/output/L

INFO:tensorflow:  name = bert_2/encoder/layer_5/output/dense/kernel:0, shape = (3072, 768)
INFO:tensorflow:  name = bert_2/encoder/layer_5/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_5/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_5/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_6/attention/self/query/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_2/encoder/layer_6/attention/self/query/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_6/attention/self/key/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_2/encoder/layer_6/attention/self/key/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_6/attention/self/value/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_2/encoder/layer_6/attention/self/value/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_6/attention/output/dense/kernel:0, s

INFO:tensorflow:  name = bert_2/encoder/layer_11/attention/self/value/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_2/encoder/layer_11/attention/self/value/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_11/attention/output/dense/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_2/encoder/layer_11/attention/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_11/attention/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_11/attention/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_11/intermediate/dense/kernel:0, shape = (768, 3072)
INFO:tensorflow:  name = bert_2/encoder/layer_11/intermediate/dense/bias:0, shape = (3072,)
INFO:tensorflow:  name = bert_2/encoder/layer_11/output/dense/kernel:0, shape = (3072, 768)
INFO:tensorflow:  name = bert_2/encoder/layer_11/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_2/encoder/layer_

INFO:tensorflow:  name = bert_3/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_3/encoder/layer_4/attention/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_3/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert_3/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert_3/encoder/layer_4/intermediate/dense/kernel:0, shape = (768, 3072)
INFO:tensorflow:  name = bert_3/encoder/layer_4/intermediate/dense/bias:0, shape = (3072,)
INFO:tensorflow:  name = bert_3/encoder/layer_4/output/dense/kernel:0, shape = (3072, 768)
INFO:tensorflow:  name = bert_3/encoder/layer_4/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_3/encoder/layer_4/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert_3/encoder/layer_4/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert_3/encoder/layer_5/attention/self/query/

INFO:tensorflow:  name = bert_3/encoder/layer_9/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert_3/encoder/layer_9/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert_3/encoder/layer_10/attention/self/query/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_3/encoder/layer_10/attention/self/query/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_3/encoder/layer_10/attention/self/key/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_3/encoder/layer_10/attention/self/key/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_3/encoder/layer_10/attention/self/value/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_3/encoder/layer_10/attention/self/value/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_3/encoder/layer_10/attention/output/dense/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert_3/encoder/layer_10/attention/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert_3/encoder/layer_10/attent

In [44]:
res = sess.run(layers[-1], {input_ids_tf: ids, input_mask_tf: mask })

In [45]:
res.shape

(1, 30, 768)

In [29]:
# estimator = tf.contrib.tpu.TPUEstimator(
#       use_tpu=False,
#       model_fn=model_fn,
#       config=run_config,
#       predict_batch_size=pred_batch_size)

INFO:tensorflow:Using config: {'_model_dir': '/var/folders/gc/2zw0lp9j6qv04bmnvn4s8gvc0000gn/T/tmp_je28r9l', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x139ec7c50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=1, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=Non

In [33]:
input_fn = input_fn_builder(
      features=features, seq_length=max_seq_len)

In [42]:
emb = None
layer = 1
for result in estimator.predict(input_fn, yield_single_examples=True):
      unique_id = int(result["unique_id"])
      feature = unique_id_to_feature[unique_id]
      emb = result["layer_output_%d" % layer]
#       output_json = collections.OrderedDict()
#       output_json["linex_index"] = unique_id
#       all_features = []
#       for (i, token) in enumerate(feature.tokens):
#         all_layers = []
#         result["layer_output_%d" % layer]
#         for (j, layer_index) in enumerate(layer_indexes):
#           layer_output = result["layer_output_%d" % j]
#           layers = collections.OrderedDict()
#           layers["index"] = layer_index
#           layers["values"] = [
#               round(float(x), 6) for x in layer_output[i:(i + 1)].flat
#           ]
#           all_layers.append(layers)
#         features = collections.OrderedDict()
#         features["token"] = token
#         features["layers"] = all_layers
#         all_features.append(features)
#       output_json["features"] = all_features

INFO:tensorflow:Could not find trained model in model_dir: /var/folders/gc/2zw0lp9j6qv04bmnvn4s8gvc0000gn/T/tmp_je28r9l, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (119547, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  

INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CK

INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:te

In [44]:
emb

array([[ 1.0143932 ,  0.76429874,  0.2510243 , ...,  1.4029906 ,
         2.0920823 , -0.286833  ],
       [ 0.63075083,  0.8088589 ,  0.22352879, ...,  1.5535302 ,
         1.6753404 , -0.08963581],
       [ 0.8191433 ,  0.602094  , -0.20782575, ..., -0.14446367,
         0.796163  , -0.03472545],
       ...,
       [ 0.94601655,  1.0988939 ,  0.14782979, ...,  1.1505815 ,
         0.91772157, -0.3733878 ],
       [ 0.87083447,  1.0458816 ,  0.13399222, ...,  1.1008292 ,
         0.9747974 , -0.37959173],
       [ 0.79682446,  1.0526316 ,  0.15087323, ...,  1.1475029 ,
         0.96538675, -0.4262566 ]], dtype=float32)

In [4]:
import tensorflow as tf
sess = tf.Session()
import numpy as np
def denseNDArrayToSparseTensor(arr, ignore_val):
  idx  = np.where(arr != ignore_val)
  return tf.SparseTensor(np.vstack(idx).T, arr[idx], arr.shape)
# dense = np.array([[[3,3,5,6,7],[0,0,0,0,0]],[[3,3,5,6,7],[0,0,0,0,0]],[[3,3,5,6,7],[0,0,0,0,0]]])
dense= np.array([[1,2],[3,4]])
print(dense.shape)
a = denseNDArrayToSparseTensor(dense,0)
c = np.array(list(range(10)), dtype=np.float32).reshape(-1,1) * np.array(list(range(20)),dtype=np.float32).reshape(1,-1)
# print(c)
# v = tf.get_variable("fgdfdfdsfwdfsg", initializer=c, dtype=tf.float32)
b = tf.nn.embedding_lookup_sparse(v, a, None, combiner=None)
s = tf.sparse.slice(a,[0,0], [2,2])
print(s)
t = tf.sparse.to_dense(s)
sess.run(tf.global_variables_initializer())
z = sess.run(t)
print(z.shape)
print(z)

(2, 2)
SparseTensor(indices=Tensor("SparseSlice_2:0", shape=(?, 2), dtype=int64), values=Tensor("SparseSlice_2:1", shape=(?,), dtype=int64), dense_shape=Tensor("SparseSlice_2:2", shape=(2,), dtype=int64))
(2, 2)
[[1 2]
 [3 4]]


In [8]:
a = np.array([1,2,3])
np.where((a !=2) & (a != 3))

(array([0]),)